In [47]:
import pandas as pd

item_info = pd.read_csv("../item_info.csv")
trx_table = pd.read_csv("../trx_table.csv")

In [48]:
def create_full_df(trx_table,item_info):
    trx_table["transaction_time"] = pd.to_datetime(trx_table["transaction_time"])
    trx_table["year"] = trx_table["transaction_time"].dt.year
    trx_table["month"] = trx_table["transaction_time"].dt.month
    trx_table["day"] = trx_table["transaction_time"].dt.day

    trx_table["hour"] = trx_table["transaction_time"].dt.hour
    trx_table["minute"] = trx_table["transaction_time"].dt.minute
    trx_table["second"] = trx_table["transaction_time"].dt.second


    trx_table.drop("second", axis=1, inplace=True)
    
    full_df = pd.merge(trx_table, item_info, on="item code", how="left")

    full_df["department_item_category"] = full_df["department"] + "-" + full_df["item_category"]
    return full_df

In [18]:
full_df = create_full_df(trx_table,item_info)

# save the full_df into csv in saved_data folder
full_df.to_csv("../synthesized_data/full_df.csv", index=False)

In [19]:
df = full_df.copy()

#################################################################################################################################

## agg_df

In [20]:
def create_agg_df_department_item_category(df):
    agg_df_department_item_category = df.groupby(['customer_code', 'department_item_category']).agg(
        total_sales_quantity=('sales_quantity', 'sum'),
        transaction_count=('transaction_time', 'count')
    ).reset_index()


    return agg_df_department_item_category

agg_df_department_item_category = create_agg_df_department_item_category(df)

agg_df_department_item_category

,customer_code,department_item_category,total_sales_quantity,transaction_count
0,customer_code_1,Hygiene-baby_needs,2,1
1,customer_code_1,Hygiene-beauty_and_personal_care,28,6
2,customer_code_1,animal products-frozen_meat,61,11
3,customer_code_1,animal products-seafood,12,3
4,customer_code_1,fresh-fruits,527,64
...,...,...,...,...
38846,customer_code_999,fresh-fruits,183,74
38847,customer_code_999,fresh-vegetables,149,50
38848,customer_code_999,homeware-stationery,22,3
38849,customer_code_999,household-baby_needs,1,1


In [26]:
def create_agg_df_item_name(df):
    agg_df_department_item_category = df.groupby(['customer_code', 'item_name']).agg(
        total_sales_quantity=('sales_quantity', 'sum'),
        transaction_count=('transaction_time', 'count')
    ).reset_index()


    return agg_df_department_item_category

agg_df_item_name = create_agg_df_item_name(df)

agg_df_item_name

,customer_code,item_name,total_sales_quantity,transaction_count
0,customer_code_1,baby_diapers,2,1
1,customer_code_1,banana,65,9
2,customer_code_1,beetroot,93,13
3,customer_code_1,body_wash,3,1
4,customer_code_1,capsicum,99,10
...,...,...,...,...
140639,customer_code_999,shampoo,9,1
140640,customer_code_999,silicon pacifiers,1,1
140641,customer_code_999,soap,4,1
140642,customer_code_999,sunscreen,2,2


In [32]:
def create_agg_df_item_category(df):
    agg_df_item_category = df.groupby(['customer_code', 'item_category']).agg(
        total_sales_quantity=('sales_quantity', 'sum'),
        transaction_count=('transaction_time', 'count')
    ).reset_index()

    return agg_df_item_category

agg_df_item_category = create_agg_df_item_category(df)

agg_df_item_category

,customer_code,item_category,total_sales_quantity,transaction_count
0,customer_code_1,baby_needs,3,2
1,customer_code_1,beauty_and_personal_care,28,6
2,customer_code_1,frozen_meat,61,11
3,customer_code_1,fruits,527,64
4,customer_code_1,seafood,12,3
...,...,...,...,...
34646,customer_code_999,fruits,183,74
34647,customer_code_999,seafood,12,3
34648,customer_code_999,stationery,22,3
34649,customer_code_999,vegetables,149,50


################################################################################################################################################

### Breakdown percentages

In [38]:
df = agg_df_department_item_category.copy()

def get_total_sales_quantity_breakdown_by_department_item_category(agg_df_department_item_category):
    # Calculate the percentage of total_sales_quantity for each department_item_category relative to the total for each customer
    df['total_sales_percentage'] = df.groupby('customer_code')['total_sales_quantity'].transform(lambda x: x / x.sum() * 100)

    # Create the total_sales_quantity_breakdown column as a dictionary
    df['total_sales_quantity_breakdown'] = df.apply(lambda row: {row['department_item_category']: f"{row['total_sales_percentage']:.2f}%"} , axis=1)

    # Aggregate the dictionaries for each customer
    result_df = df.groupby('customer_code')['total_sales_quantity_breakdown'].apply(
        lambda x: {k: v for d in x for k, v in d.items()}
    ).reset_index()


    return result_df

total_sales_quantity_breakdown_by_department_item_category_df = get_total_sales_quantity_breakdown_by_department_item_category(df)
# save the total_sales_quantity_breakdown_by_department_item_category_df into csv in saved_data folder

total_sales_quantity_breakdown_by_department_item_category_df.to_csv("../synthesized_data/total_sales_quantity_breakdown_by_department_item_category_df.csv", index=False)

In [39]:
df = agg_df_item_category.copy()

def get_total_sales_quantity_breakdown_by_item_category(df):
    # Calculate the percentage of total_sales_quantity for each department_item_category relative to the total for each customer
    df['total_sales_percentage'] = df.groupby('customer_code')['total_sales_quantity'].transform(lambda x: x / x.sum() * 100)

    # Create the total_sales_quantity_breakdown column as a dictionary
    df['total_sales_quantity_breakdown'] = df.apply(lambda row: {row['item_category']: f"{row['total_sales_percentage']:.2f}%"} , axis=1)

    # Aggregate the dictionaries for each customer
    result_df = df.groupby('customer_code')['total_sales_quantity_breakdown'].apply(
        lambda x: {k: v for d in x for k, v in d.items()}
    ).reset_index()


    return result_df

total_sales_quantity_breakdown_by_item_category_df = get_total_sales_quantity_breakdown_by_item_category(df)

# save the total_sales_quantity_breakdown_by_item_category_df into csv in saved_data folder
total_sales_quantity_breakdown_by_item_category_df.to_csv("../synthesized_data/total_sales_quantity_breakdown_by_item_category_df.csv", index=False)

In [40]:
agg_df_item_name

,customer_code,item_name,total_sales_quantity,transaction_count
0,customer_code_1,baby_diapers,2,1
1,customer_code_1,banana,65,9
2,customer_code_1,beetroot,93,13
3,customer_code_1,body_wash,3,1
4,customer_code_1,capsicum,99,10
...,...,...,...,...
140639,customer_code_999,shampoo,9,1
140640,customer_code_999,silicon pacifiers,1,1
140641,customer_code_999,soap,4,1
140642,customer_code_999,sunscreen,2,2


In [42]:
df = agg_df_item_name.copy()

def get_total_sales_quantity_breakdown_by_item_name(df):
    # Calculate the percentage of total_sales_quantity for each department_item_category relative to the total for each customer
    df['total_sales_percentage'] = df.groupby('customer_code')['total_sales_quantity'].transform(lambda x: x / x.sum() * 100)

    # Create the total_sales_quantity_breakdown column as a dictionary
    df['total_sales_quantity_breakdown'] = df.apply(lambda row: {row['item_name']: f"{row['total_sales_percentage']:.2f}%"} , axis=1)

    # Aggregate the dictionaries for each customer
    result_df = df.groupby('customer_code')['total_sales_quantity_breakdown'].apply(
        lambda x: {k: v for d in x for k, v in d.items()}
    ).reset_index()


    return result_df

total_sales_quantity_breakdown_by_item_name_df = get_total_sales_quantity_breakdown_by_item_name(df)

# save the total_sales_quantity_breakdown_by_item_name_df into csv in saved_data folder
total_sales_quantity_breakdown_by_item_name_df.to_csv("../synthesized_data/total_sales_quantity_breakdown_by_item_name_df.csv", index=False)

In [43]:
data = full_df.copy()

# Calculate the total sales quantity and transaction count for each customer in each department
customer_department_sales = data.groupby(['customer_code', 'department_item_category','month']).agg({'sales_quantity': 'sum', 'transaction_time': 'count'})

# Calculate the volume as total sales_quantity divided by transaction count
customer_department_sales['volume'] = customer_department_sales['sales_quantity'] / customer_department_sales['transaction_time']
customer_department_sales.reset_index(inplace=True)


In [44]:
customer_department_sales

,customer_code,department_item_category,month,sales_quantity,transaction_time,volume
0,customer_code_1,Hygiene-baby_needs,5,2,1,2.0
1,customer_code_1,Hygiene-beauty_and_personal_care,1,5,2,2.5
2,customer_code_1,Hygiene-beauty_and_personal_care,4,10,2,5.0
3,customer_code_1,Hygiene-beauty_and_personal_care,5,13,2,6.5
4,customer_code_1,animal products-frozen_meat,1,10,2,5.0
...,...,...,...,...,...,...
143071,customer_code_999,homeware-stationery,4,8,1,8.0
143072,customer_code_999,household-baby_needs,3,1,1,1.0
143073,customer_code_999,lifestyle-wellness_food,2,6,1,6.0
143074,customer_code_999,lifestyle-wellness_food,3,3,1,3.0


In [45]:
# sort by volume
customer_department_sales.sort_values(by=['customer_code', 'volume'], ascending=[True, False], inplace=True)
customer_department_sales

# get the top 3 department_item_category for each customer per each month
top3_department_item_category = customer_department_sales.groupby(['customer_code', 'month']).head(3)

# save the top3_department_item_category into csv in saved_data folder
top3_department_item_category.to_csv("../synthesized_data/top3_department_item_category.csv", index=False)

In [46]:
# get the bottom 3 department_item_category for each customer per each month
bottom3_department_item_category = customer_department_sales.groupby(['customer_code', 'month']).tail(3)

# save the bottom3_department_item_category into csv in saved_data folder
bottom3_department_item_category.to_csv("../synthesized_data/bottom3_department_item_category.csv", index=False)

In [178]:
# GET RESULTS FOR CUSTOMER 1 IN MONTH 7
top3_department_item_category[(top3_department_item_category['customer_code'] == 'customer_code_1') & (top3_department_item_category['month'] == 1)]

,customer_code,department_item_category,month,sales_quantity,transaction_time,volume
4,customer_code_1,animal products-frozen_meat,1,10,2,5.000000
12,customer_code_1,fresh-fruits,1,58,12,4.833333
19,customer_code_1,fresh-vegetables,1,32,7,4.571429


In [177]:
bottom3_department_item_category[(bottom3_department_item_category['customer_code'] == 'customer_code_1') & (bottom3_department_item_category['month'] == 1)]

,customer_code,department_item_category,month,sales_quantity,transaction_time,volume
12,customer_code_1,fresh-fruits,1,58,12,4.833333
19,customer_code_1,fresh-vegetables,1,32,7,4.571429
1,customer_code_1,Hygiene-beauty_and_personal_care,1,5,2,2.500000


In [148]:

# Define high-volume buyers for each department as those whose volume is above the median volume within the department
median_volume_per_department = customer_department_sales.groupby('department_item_category')['volume'].median()
median_volume_per_department

department_item_category
Hygiene-baby_needs                   2.000000
Hygiene-beauty_and_personal_care     4.666667
animal products-dairy                7.898148
animal products-frozen_meat          5.545455
animal products-seafood              4.000000
fresh-fruits                         5.777778
fresh-vegetables                     4.714286
homeware-stationery                  7.333333
household-baby_needs                 1.000000
household-wellness_products         10.797619
lifestyle-wellness_food              5.000000
pet_products-pet_care                3.510488
pet_products-pet_food                3.473684
Name: volume, dtype: float64

In [142]:

customer_department_sales['High-volume vs. low-volume buyers'] = customer_department_sales.apply(
    lambda x: 'High-volume' if x['volume'] > median_volume_per_department[x.name[1]] else 'Low-volume', axis=1
)



TypeError: 'int' object is not subscriptable